Для проведения атаки отравлением набора обучим нейронную сеть VoxCeleb на базе части датасета. Для примера из общего датасета были взяты только данные 4 пользователей. Это было сделано для того, чтобы обучение не проходило слишком долго. Частичный датасет представлен в виде папки poison_data и содержит в себе папку dataset, содержащую непосредственно  данные и файлы train_list.txt и test_list.txt, содержащие списки тестовых и тренировочных данных.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip './drive/MyDrive/Colab_Notebooks/Project_5.2/poison_data.zip'

Archive:  ./drive/MyDrive/Colab_Notebooks/Project_5.2/poison_data.zip
   creating: poison_data/dataset/
   creating: poison_data/dataset/id10283/
   creating: poison_data/dataset/id10283/52SYuI0cCbQ/
  inflating: poison_data/dataset/id10283/52SYuI0cCbQ/00001.wav  
  inflating: poison_data/dataset/id10283/52SYuI0cCbQ/00002.wav  
  inflating: poison_data/dataset/id10283/52SYuI0cCbQ/00003.wav  
  inflating: poison_data/dataset/id10283/52SYuI0cCbQ/00004.wav  
   creating: poison_data/dataset/id10283/arklnCzCq48/
  inflating: poison_data/dataset/id10283/arklnCzCq48/00001.wav  
  inflating: poison_data/dataset/id10283/arklnCzCq48/00002.wav  
  inflating: poison_data/dataset/id10283/arklnCzCq48/00003.wav  
   creating: poison_data/dataset/id10283/clznvDi-ybg/
  inflating: poison_data/dataset/id10283/clznvDi-ybg/00001.wav  
  inflating: poison_data/dataset/id10283/clznvDi-ybg/00002.wav  
  inflating: poison_data/dataset/id10283/clznvDi-ybg/00003.wav  
  inflating: poison_data/dataset/id10283/c

Загружаем код проекта VoxCeleb с GitHub.

In [ ]:
!git clone https://github.com/clovaai/voxceleb_trainer

Cloning into 'voxceleb_trainer'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 287 (delta 32), reused 65 (delta 29), pack-reused 204
Receiving objects: 100% (287/287), 110.10 KiB | 2.34 MiB/s, done.
Resolving deltas: 100% (146/146), done.


сменим рабочую директорию на папку загруженного проекта

In [ ]:
%cd voxceleb_trainer/

/content/voxceleb_trainer


Прежде чем запустить обучение нейронной сети изменим стандартный конфигурационный файл ResNetSE34L_AM.yaml, содержащийся в папке configs, так, чтобы он принял следующий вид:
<br>
<br>model: ResNetSE34L
<br>log_input: True
<br>encoder_type: SAP
<br>trainfunc: amsoftmax
<br>save_path: exps/ResNetSE34L_AM
<br>nClasses: 5994
<br>batch_size: 2
<br>scale: 30
<br>margin: 0.3
<br>max_epoch: 50
<br>train_list: "путь к файлу train_list.txt"
<br>test_list: "путь к файлу test_list.txt"
<br>train_path: "путь к папке dataset"
<br>test_path: "путь к папке dataset"

In [ ]:
!python ./trainSpeakerNet.py --config ./configs/ResNetSE34L_AM.yaml 

Python Version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
PyTorch Version: 1.12.1+cu113
Number of GPUs: 1
Save path: exps/ResNetSE34L_AM
Embedding size is 512, encoder SAP.
Initialised AMSoftmax m=0.300 s=30.000
Initialised Adam optimizer
Initialised step LR scheduler
Processing 702 of 702:Loss 10.705543 TEER/TAcc 0.000% - 44.07 Hz 
 2022-10-29 07:40:50 Epoch 1, TEER/TAcc 0.00, TLOSS 10.705543, LR 0.001000
Processing 704 of 704:Loss 10.132185 TEER/TAcc 0.710% - 44.48 Hz 
 2022-10-29 07:41:07 Epoch 2, TEER/TAcc 0.71, TLOSS 10.132185, LR 0.001000
Processing 704 of 704:Loss 9.788436 TEER/TAcc 3.267% - 45.31 Hz 
 2022-10-29 07:41:24 Epoch 3, TEER/TAcc 3.27, TLOSS 9.788436, LR 0.001000
Processing 716 of 716:Loss 9.137428 TEER/TAcc 8.659% - 42.34 Hz 
 2022-10-29 07:41:41 Epoch 4, TEER/TAcc 8.66, TLOSS 9.137428, LR 0.001000
Processing 720 of 720:Loss 8.551002 TEER/TAcc 16.389% - 47.48 Hz 
 2022-10-29 07:41:58 Epoch 5, TEER/TAcc 16.39, TLOSS 8.551002, LR 0.001000
Processing 696 of 6

В рабочей папке появилась папка exps, содержащая в себе результаты обучения. Обученные модели хранятся в папке ResNetSE34L_AM/model, самая актуальная из них - model000000050.model
<br>Протестируем данную модель:

In [ ]:
from torch import threshold
import numpy as np
from tuneThreshold import *
from SpeakerNet import *
from DatasetLoader import *
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

def calculate_threshold(sc, lab):
    fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
    _, threshold = ComputeMinDcf(fnrs, fprs, thresholds, 0.05, 1, 1)
    return threshold


def calc_accuracy(scores, labels, threshold):
    predictions = (np.array(scores) > threshold).astype(int)
    labels = np.array(labels)
    accuracy = sum(predictions == labels) / len(predictions) * 100
    print('accuracy: ', accuracy, '%')
    return accuracy


def calc_errors(scores, labels, threshold):
    predictions = (np.array(scores) > threshold).astype(int)
    labels = np.array(labels)
    first = 0
    second = 0
    difference = np.where(predictions != labels)[0]
    for diff in difference:
        if labels[diff] == 1:
            first += 1
        else:
            second += 1

    first /= len(predictions)
    second /= len(predictions)
    print('first = ', first * 100, '%; second = ', second * 100, '%')
    return first * 100, second * 100

def main():
    global trainer

    s = SpeakerNet('ResNetSE34L', 'adam', 'angleproto', nPerSpeaker=400, nOut=512)
    s = WrappedModel(s).cuda(0)
    trainer = ModelTrainer(s, 'adam', 'steplr', gpu=0, mixedprec=False, lr=0.001, lr_decay=0.95, weight_decay=0,
                           test_interval=10, max_epoch=500)
    trainer.loadParameters('exps/ResNetSE34L_AM/model/model000000050.model')
    trainer.__model__.eval()

    sc, lab, _ = trainer.evaluateFromList(
        test_list='/content/poison_data/test_list.txt',
        #test_list='путь к for_test_poison.txt',
        test_path='/content/poison_data/dataset',
        nDataLoaderThread=5,
        distributed=False,
        print_interval=100,
        num_eval=10,
        eval_frames=400)
    
    print("")
    threshold = calculate_threshold(sc, lab)
    calc_accuracy(sc, lab, threshold)
    calc_errors(sc, lab, threshold)


if __name__ == '__main__':
    main()

Embedding size is 512, encoder SAP.
Initialised AngleProto
Initialised Adam optimizer
Initialised step LR scheduler
__L__.W is not in the model.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Reading 100 of 172: 29.02 Hz, embedding size 512
Computing 0 of 100: 0.00 Hz
accuracy:  100.0 %
first =  0.0 %; second =  0.0 %


Создадим копию конфигурационного файла для проведения атаки

In [ ]:
!cp configs/ResNetSE34L_AM.yaml configs/ResNetSE34L_AM_poisoned.yaml

Прежде чем запустить обучение нейронной сети изменим скопированный конфигурационный файл ResNetSE34L_AM_poisoned.yaml, содержащийся в папке configs, так, чтобы он ссылался уже на файлы с искаженными метками:
<br>
<br>model: ResNetSE34L
<br>log_input: True
<br>encoder_type: SAP
<br>trainfunc: amsoftmax
<br>save_path: exps/ResNetSE34L_AM_poisoning
<br>nClasses: 5994
<br>batch_size: 2
<br>scale: 30
<br>margin: 0.3
<br>max_epoch: 50
<br>train_list: "путь к файлу train_list_poisoning.txt"
<br>test_list: "путь к файлу test_list_poisoning.txt"
<br>train_path: "путь к папке dataset"
<br>test_path: "путь к папке dataset"

In [ ]:
!python ./trainSpeakerNet.py --config ./configs/ResNetSE34L_AM_poisoned.yaml

Python Version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
PyTorch Version: 1.12.1+cu113
Number of GPUs: 1
Save path: exps/ResNetSE34L_AM_poisoning
Embedding size is 512, encoder SAP.
Initialised AMSoftmax m=0.300 s=30.000
Initialised Adam optimizer
Initialised step LR scheduler
Processing 698 of 698:Loss 10.684199 TEER/TAcc 0.000% - 47.49 Hz 
 2022-10-29 07:59:00 Epoch 1, TEER/TAcc 0.00, TLOSS 10.684199, LR 0.001000
Processing 708 of 708:Loss 10.245736 TEER/TAcc 0.000% - 47.60 Hz 
 2022-10-29 07:59:17 Epoch 2, TEER/TAcc 0.00, TLOSS 10.245736, LR 0.001000
Processing 712 of 712:Loss 9.740713 TEER/TAcc 2.247% - 41.77 Hz 
 2022-10-29 07:59:33 Epoch 3, TEER/TAcc 2.25, TLOSS 9.740713, LR 0.001000
Processing 718 of 718:Loss 9.192299 TEER/TAcc 10.585% - 48.07 Hz 
 2022-10-29 07:59:50 Epoch 4, TEER/TAcc 10.58, TLOSS 9.192299, LR 0.001000
Processing 726 of 726:Loss 8.629392 TEER/TAcc 14.738% - 45.66 Hz 
 2022-10-29 08:00:07 Epoch 5, TEER/TAcc 14.74, TLOSS 8.629392, LR 0.001000
Process

In [ ]:
from torch import threshold
import numpy as np
from tuneThreshold import *
from SpeakerNet import *
from DatasetLoader import *
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

def calculate_threshold(sc, lab):
    fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
    _, threshold = ComputeMinDcf(fnrs, fprs, thresholds, 0.05, 1, 1)
    return threshold


def calc_accuracy(scores, labels, threshold):
    predictions = (np.array(scores) > threshold).astype(int)
    labels = np.array(labels)
    accuracy = sum(predictions == labels) / len(predictions) * 100
    print('accuracy: ', accuracy, '%')
    return accuracy


def calc_errors(scores, labels, threshold):
    predictions = (np.array(scores) > threshold).astype(int)
    labels = np.array(labels)
    first = 0
    second = 0
    difference = np.where(predictions != labels)[0]
    for diff in difference:
        if labels[diff] == 1:
            first += 1
        else:
            second += 1

    first /= len(predictions)
    second /= len(predictions)
    print('first = ', first * 100, '%; second = ', second * 100, '%')
    return first * 100, second * 100

def main():
    global trainer

    s = SpeakerNet('ResNetSE34L', 'adam', 'angleproto', nPerSpeaker=400, nOut=512)
    s = WrappedModel(s).cuda(0)
    trainer = ModelTrainer(s, 'adam', 'steplr', gpu=0, mixedprec=False, lr=0.001, lr_decay=0.95, weight_decay=0,
                           test_interval=10, max_epoch=500)
    trainer.loadParameters('exps/ResNetSE34L_AM_poisoning/model/model000000050.model')
    trainer.__model__.eval()

    sc, lab, _ = trainer.evaluateFromList(
        test_list='/content/poison_data/test_list.txt',
        #test_list='путь к for_test_poison.txt',
        test_path='/content/poison_data/dataset',
        nDataLoaderThread=5,
        distributed=False,
        print_interval=100,
        num_eval=10,
        eval_frames=400)
    
    print("")
    threshold = calculate_threshold(sc, lab)
    calc_accuracy(sc, lab, threshold)
    calc_errors(sc, lab, threshold)


if __name__ == '__main__':
    main()

Embedding size is 512, encoder SAP.
Initialised AngleProto
Initialised Adam optimizer
Initialised step LR scheduler
__L__.W is not in the model.
Reading 100 of 172: 53.31 Hz, embedding size 512
Computing 0 of 100: 0.00 Hz
accuracy:  99.0 %
first =  1.0 %; second =  0.0 %
